In [ ]:
import boto3
import numpy as np

# Create a SageMaker client
sagemaker = boto3.client("sagemaker")

# Define the S3 bucket and prefix
bucket = "your_bucket"
prefix = "your_prefix"

# Define the function handler
def lambda_handler(event, context):
    # Get the object key from the event
    key = event["Records"][0]["s3"]["object"]["key"]
    
    # Load the dataframe from S3
    df = pd.read_csv(f"s3://{bucket}/{key}")
    
    # Split it into 2000 row chunks
    chunks = np.array_split(df, len(df) // 2000 + 1)
    
    # Loop through the chunks and send them to SageMaker for batch transform
    for i, chunk in enumerate(chunks):
        # Save the chunk as a csv file in S3
        chunk.to_csv(f"s3://{bucket}/{prefix}/chunk_{i}.csv", index=False)
        
        # Create a batch transform request
        request = {
            "TransformJobName": f"chunk_{i}",
            "ModelName": "your_model_name",
            "TransformInput": {
                "DataSource": {
                    "S3DataSource": {
                        "S3DataType": "S3Prefix",
                        "S3Uri": f"s3://{bucket}/{prefix}/chunk_{i}.csv"
                    }
                },
                "ContentType": "text/csv",
                "SplitType": "Line"
            },
            "TransformOutput": {
                "S3OutputPath": f"s3://{bucket}/{prefix}/output_{i}"
            },
            "TransformResources": {
                "InstanceType": "ml.m4.xlarge",
                "InstanceCount": 1
            }
        }
        
        # Send the request to SageMaker
        response = sagemaker.create_transform_job(**request)
        
        # Print the response
        print(response)
